In [ ]:
import os
import glob
import pickle
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

fontsize = 8
lw = 0.75

matplotlib.rc('font', **{'family': 'Times New Roman', 'size': fontsize})
matplotlib.rc('axes', **{'linewidth': 0.75, 'labelsize': fontsize})
matplotlib.rc('xtick', **{'labelsize': fontsize})
matplotlib.rc('ytick', **{'labelsize': fontsize})
matplotlib.rc('xtick.major', **{'width': lw, 'size':3})
matplotlib.rc('ytick.major', **{'width': lw, 'size':3})
matplotlib.rc('ytick.minor', **{'width': lw, 'size':1.5})

In [ ]:
def compute_score(time, measurement, prediction):
    err = np.abs(measurement - prediction)
    std = np.std(err)
    idx = (time >= 7) & (time <= 20)
    err[idx] *= 2
    mae = np.mean(np.concatenate([err, np.sort(err)[-12:] * 4]))
    return 5 * mae + std

In [ ]:
def plot_prediction(future, with_generation, with_temperature, stop=5, ylim=[0.8, 8.2], ax=None, lw=1):
    filename_template = f'*{future:.2f}_hours-consumption'
    if with_generation:
        filename_template += '-generation'
    if with_temperature:
        filename_template += '-temperature'
    pkl_files = sorted(glob.glob(filename_template + '.pkl'))
    suffix = '-'.join(os.path.splitext(pkl_files[0])[0].split('-')[1:])
    print(pkl_files)
    data = [pickle.load(open(pkl_file, 'rb')) for pkl_file in pkl_files]
    
    if ax is None:
        ax = plt.gca()

    for i,today in enumerate(data[:-1]):
        if i == 0:
            ax.plot(today['t_past'] + i * 24, today['past_measured'], 'k', lw=lw, label='Measured')
        else:
            ax.plot(today['t_past'] + i * 24, today['past_measured'], 'k', lw=lw)
        if i >= stop:
            break
    if i+1 <= stop:
        tomorrow = data[i+1]
        ax.plot(tomorrow['t_past'] + (i+1) * 24, tomorrow['past_measured'], 'k', lw=lw)
    scores = np.zeros(len(data)-1)
    mapes = np.zeros(len(data)-1)
    for i,(today,tomorrow) in enumerate(zip(data[:-1], data[1:])):
        if i == 0:
            ax.plot(today['t_future'] + i * 24, today['future_predicted'], 'r', lw=lw, label='Predicted')
        else:
            ax.plot(today['t_future'] + i * 24, today['future_predicted'], 'r', lw=lw)
        scores[i] = compute_score(today['t_future'], tomorrow['past_measured'], today['future_predicted'])
        mapes[i] = 100 * np.mean(np.abs((tomorrow['past_measured'] - today['future_predicted']) / \
                                         tomorrow['past_measured']))
        #print(f'Score = {score:5.2f}  MAPE = {mape:5.2f}%')
        if i >= stop:
            break
    if i <= stop:
        ax.plot(tomorrow['t_future'] + (i+1) * 24, tomorrow['future_predicted'], 'r', lw=lw)
    return scores,mapes

In [ ]:
row_height, col_width = 1., 2.25  # [inches]
n_rows = 3
n_cols = 1
if n_cols == 1:
    col_width = 3
fig,ax = plt.subplots(n_rows, n_cols, figsize=(col_width * n_cols, row_height * n_rows), squeeze=False)

ylim = [0.8, 8.2]
stop = 5
with_generation = (False, True, True)
with_temperature = (False, False, True)
scores = {}
mapes = {}
YN = {False: 'N', True: 'Y'}
future_sizes =  0.25, 0.5, 1.0
for j, future in enumerate(future_sizes[:n_cols]):
    scores[future] = {}
    mapes[future] = {}
    for i in range(n_rows):
        a = ax[i,j]
        s,m = plot_prediction(future, with_generation[i], with_temperature[i], \
                              stop, ylim, ax=a, lw=0.75)
        key = f'Y{YN[with_generation[i]]}{YN[with_temperature[i]]}'
        scores[future][key] = s
        mapes[future][key] = m
        for side in 'right','top':
            a.spines[side].set_visible(False)
        a.grid(which='major', axis='both', color=[.6,.6,.6], lw=0.5, linestyle=':')

        a.set_xlim([-26, 122])
        a.set_xticks(np.r_[-24 : 121 : 24])
        a.set_xticklabels([])
        a.xaxis.set_minor_locator(matplotlib.ticker.AutoMinorLocator(2))
        
        a.set_ylim(ylim)
        a.set_yticks(np.r_[1 : 8.5 : 2])
        a.set_yticklabels([])
        a.yaxis.set_minor_locator(matplotlib.ticker.AutoMinorLocator(2))

if n_cols > 1:
    for i,a in enumerate(ax[0,:]):
        title = f'Future: {2**i*15:.0f} min'
        a.set_title(title)

for a in ax[-1,:]:
    a.set_xticklabels(np.r_[-24 : 121 : 24])
    a.set_xlabel('Time [hours]')

for a in ax[:,0]:
    a.set_yticklabels(np.r_[1 : 8.5 : 2])
ax[1,0].set_ylabel('Consumption [kW]')

fig.tight_layout(pad=0.25)
# if n_cols == 1:
#     fig.savefig('predictions_15_min_future.pdf')
# else:
#     fig.savefig('all_predictions.pdf')

In [ ]:
data = scores
measure_name = 'SCORE'
units = ''
X = np.array([measures for D in data.values() for measures in D.values()])
best = X.argmin(axis=0)
worst = X.argmax(axis=0)
week = X.mean(axis=1)
row = 0
print(r'\begin{table}[tb!!]')
print(r'\begin{center}')
print(r'\caption{\ldots}')
print(r'\label{tab:}')
print(r'\begin{tabular}{c|ccc|ccccc|c}')
#print(r'{} & {} & {} & {} & \multicolumn{5}{c}{' + measure_name + r'} \\')
#print(r'\hline')
print(r'Future & C & G & T & Mon & Tue & Wed & Thu & Fri & Avg. \\')
print(r'\hline')
for future,D in data.items():
    for key,measures in D.items():
        sys.stdout.write(f'{future*60:.0f} min')
        for yn in key:
            sys.stdout.write(f' & {yn}')
        for j,meas in enumerate(measures):
            if best[j] == row:
                sys.stdout.write(f' & \\green{{{meas:4.1f}{units}}}')
                pass
            elif worst[j] == row:
                sys.stdout.write(f' &   \\red{{{meas:4.1f}{units}}}')
                pass
            else:
                sys.stdout.write(f' & {meas:12.1f}{units}')
        if week.argmin() == row:
            sys.stdout.write(f' & \\green{{{measures.mean():4.1f}{units}}} \\\\\n')
        elif week.argmax() == row:
            sys.stdout.write(f' &   \\red{{{measures.mean():4.1f}{units}}} \\\\\n')
        else:
            sys.stdout.write(f' &         {measures.mean():4.1f}{units} \\\\\n')
        row += 1
print(r'\end{tabular}')
print(r'\end{center}')
print(r'\end{table}')